# Regression

In this tutorial, we'll explore regression models using Python's scikit learn (sklearn) package and the built data set. Please keep in mind that although regression is considered one of the simplest or most basic machine learning techniques, a thorough understanding of the assumptions and limitations is essential for a correct interpretation of the results. 

We'll start by loading the linear_model and diabetes data set from sklearn. Note that we're only loading the components that we need for this exercise since the entire sklearn package is extremely large.

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import load_diabetes
import numpy as np
import matplotlib.pyplot as plt
diabetes = load_diabetes()

Let's take a look at the diabetes data set. We are interested in how the disease progression depends on factors such as age, sex, BMI (body mass index) and blood pressure. Note that these factors have been mean-centered and scaled by the standard deviation.

The disease progression is the *dependent* variable and age, sex, BMI etc. are the *independent* variables.

In [ ]:
diabetes

To make this a little more readable, we can print just the description

In [ ]:
print(diabetes.DESCR)

The linear regression fitting function (linear.fit) expects a 2D arrays for the data (# samples x # features). We'll start off by working with a single feature at a time. 

In [ ]:
# Extract column corresponding to BMI from data set and convert to (n x 1) arrays
bmi              = diabetes.data[:, np.newaxis, 2]
disease_progress = diabetes.target

Next we'll fit the model and use the model to calculate the expected disease progression. We also calculate the $R^2$ coefficient, which is the percentage of the change in the dependent variable that can be attributed to the change in the independent variable.

$R^2 = 1 - \frac{\Sigma (y - ypred)^2}{\Sigma (y - ymean)^2}$


In [ ]:
# Create and fit the model
regr = linear_model.LinearRegression()
regr.fit(bmi, disease_progress)

# Apply the model (predict the disease progression from BMI using linear model)
disease_progress_pred = regr.predict(bmi)

print('Intercept: ', regr.intercept_)
print('Coefficient: ', regr.coef_[0])
print('Variance score (R2): %.2f' % r2_score(disease_progress, disease_progress_pred))

In [ ]:
plt.scatter(bmi, disease_progress,  color='black')
plt.plot(bmi, disease_progress_pred, color='blue', linewidth=3)
plt.xlabel('BMI')
plt.ylabel('disease progression')

plt.show()

In this example, we used all the the available data to train the model. Often in machine learning applications we want to set aside some of the data to test the model. This allows us to determine if our model has predictive value.

Let's go back and divide our data into training and testing sets.

In [ ]:
bmi_train = bmi[:-20] # All but last 20 elements
bmi_test  = bmi[-20:] # Last 20 elements

disease_progress_train = disease_progress[:-20] # All but last 20 elements
disease_progress_test  = disease_progress[-20:] # Last 20 elements

In [ ]:
# Create and fit the model
regr = linear_model.LinearRegression()
regr.fit(bmi_train, disease_progress_train)

# Apply the model
disease_progress_pred = regr.predict(bmi_test)

print('Intercept: ', regr.intercept_)
print('Coefficient: ', regr.coef_[0])
print('Variance score (R2): %.2f' % r2_score(disease_progress_test, disease_progress_pred))

In [ ]:
plt.scatter(bmi_test, disease_progress_test,  color='black')
plt.plot(bmi_test, disease_progress_pred, color='blue', linewidth=3)
plt.xlabel('BMI')
plt.ylabel('disease progression')

plt.show()

## Multiple linear regression

So far, we looked at a simple linear regression model that depends on a single variable. We generalize this to a Multiple Linear Regression model of the form

$y_i = b + a_1 x1_i + a_2 x2_i + ...$

In the example below, we use all ten baseline variables to build our model

In [ ]:
# Create and fit the model
regr = linear_model.LinearRegression()
regr.fit(diabetes.data, diabetes.target)

# Apply the model (predict the disease progression from BMI using linear model)
disease_progress_pred = regr.predict(diabetes.data)

print('Intercept: ', regr.intercept_)
print('Coefficient: ', regr.coef_)
print('Variance score (R2): %.2f' % r2_score(diabetes.target, disease_progress_pred))

## Combining sklearn with pandas

So far, we've been doing things the hard way - selecting columns from our 2D array of data, saving as new arrays, making sure that we retain multiple dimensions even if we only want one column.

A much easier approach is to use pandas and work directly with data frames. This has the additional advantage that you can the dataframe methods to operate on the data. In the following examples, we train using all the independent variables, three variables and 

In [ ]:
import pandas as pd
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
disease_progress = pd.DataFrame(diabetes.target)

In [ ]:
df.head(3)

In [ ]:
disease_progress.head(3)

#### Build the multiple linear regression model using all independent variables

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(df, disease_progress)
disease_progress_pred = regr.predict(df)

print('Intercept: ', regr.intercept_)
print('Coefficient: ', regr.coef_[0])
print('Variance score (R2): %.2f' % r2_score(disease_progress, disease_progress_pred))

#### Build the multiple linear regression model using age, sex and bmi

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(df[["age", "bmi"]], disease_progress)
disease_progress_pred = regr.predict(df[["age", "bmi"]])

print('Intercept: ', regr.intercept_)
print('Coefficient: ', regr.coef_[0])
print('Variance score (R2): %.2f' % r2_score(disease_progress, disease_progress_pred))

#### Build the ordinary linear regression model using bmi - set aside data for testing

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(df[["bmi"]][:-20], disease_progress[:-20])
disease_progress_pred = regr.predict(df[["bmi"]][-20:])

print('Intercept: ', regr.intercept_)
print('Coefficient: ', regr.coef_[0])
print('Variance score (R2): %.2f' % r2_score(disease_progress[-20:], disease_progress_pred[-20:]))

#### Perform ordinary linear regression, looping over all features

In [ ]:
for feature in df.columns.get_values():
    print("Feature: ", feature)
    regr = linear_model.LinearRegression()
    regr.fit(df[[feature]], disease_progress)
    disease_progress_pred = regr.predict(df[[feature]])

    print('Intercept: ', regr.intercept_)
    print('Coefficient: ', regr.coef_[0])
    print('Variance score (R2): %.2f' % r2_score(disease_progress, disease_progress_pred))
    print()

## Using column of data frame as target

In [ ]:
import pandas as pd
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['Disease progression'] = diabetes.target

In [ ]:
df.head()

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(df[['age', 'bmi']], df[['Disease progression']])
disease_progress_pred = regr.predict(df[['age', 'bmi']])

print('Intercept: ', regr.intercept_)
print('Coefficient: ', regr.coef_[0])
print('Variance score (R2): %.2f' % r2_score(df[['Disease progression']], disease_progress_pred))

## Working with categorical variables

Keep in mind that sex is a categorical value and cannot be used for linear regression. You might recall when we looped over the features that the $R^2$ value for dependence of disease progression on sex was exactly zero. We'll need to take a different approach.

This was easy to overlook since sex appears as a numerical value in the data set. Male and female were likely assigned the values $\pm 1$, which then became 0.050680 and -0.044642 after normalizing the data.

In [ ]:
progression_male = np.array(df['Disease progression'][df['sex'] > 0])
progression_female = np.array(df['Disease progression'][df['sex'] < 0])

In [ ]:
progression_male.std()

In [ ]:
progression_female.std()

In [ ]:
from scipy import stats
stats.ttest_ind(progression_male, progression_female, equal_var=True)